In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

Creating the data. 

In [2]:
df = pd.DataFrame({
    'institution_name': [
    "Berkshire Hathaway",
    "Renaissance Technologies",
    "Citadel Advisors",
    "BlackRock Inc.",
    "Bridgewater Associates",
    "Two Sigma Investments",
    "Millennium Management",
    "Vanguard Group",
    "FMR LLC",
    "State Street Corp"
],
    'cik': [
    "0001067983",
    "0001166559",
    "0001037389",
    "0001081060",
    "0001103804",
    "0000922971",
    "0001079114",
    "0000912057",
    "0000316927",
    "0000354204" 
]
})
df['cik'] = df['cik'].astype(str).str.zfill(10)
df.to_csv('top_institutions.csv', index=False)


In [3]:

institutions_df = pd.read_csv('top_institutions.csv', dtype={'cik': str})
institutions_df['cik'] = institutions_df['cik'].str.zfill(10)

top_institutions = dict(zip(institutions_df['cik'], institutions_df['institution_name']))


In [4]:
# with date filter, just to get the recent filings. 

def fetch_filing_metadata(cik, institution_name, days_ago=70):
    cutoff_date = datetime.utcnow() - timedelta(days=days_ago)

    url = f"https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type=13F-HR&owner=exclude&count=40&output=atom"
    headers = {
        "User-Agent": "DataScienceInternshipBot/1.0 (contact: chandanarchutha.n@gmail.com)",
        "Accept": "application/xml",
        "Accept-Language": "en-US,en;q=0.5",
        "Referer": "https://www.sec.gov/"
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "xml")
    entries = soup.find_all("entry")

    filing_links = []
    filed_dates = []
    institution_names = []

    for entry in entries:
        date_str = entry.find("updated").text.strip()
        date_obj = datetime.strptime(date_str[:10], "%Y-%m-%d")  # keep only YYYY-MM-DD

        if date_obj >= cutoff_date:
            link = entry.find("link")["href"].strip()
            filing_links.append(link)
            filed_dates.append(date_str)
            institution_names.append(institution_name)

    return filing_links, filed_dates, institution_names


In [5]:
# without the day filter, fetches all the available links

def fetch_filing_metadata(cik, institution_name):
    url = f"https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type=13F-HR&owner=exclude&count=40&output=atom"
    headers = {
        "User-Agent": "DataScienceInternshipBot/1.0 (contact: chandanarchutha.n@gmail.com)",
        "Accept": "application/xml",
        "Accept-Language": "en-US,en;q=0.5",
        "Referer": "https://www.sec.gov/"
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "xml")
    entries = soup.find_all("entry")

    filing_links = []
    filed_dates = []
    institution_names = []

    for entry in entries:
        link = entry.find("link")["href"].strip()
        date = entry.find("updated").text.strip()
        filing_links.append(link)
        filed_dates.append(date)
        institution_names.append(institution_name)

    return filing_links, filed_dates, institution_names

**Extract XML links**

In [6]:
# Function to extract XML info table link from index page
def extract_info_table_xml(index_url):
    headers = {
        "User-Agent": "DataScienceInternshipBot/1.0 (contact: chandanarchutha.n@gmail.com)",
        "Accept-Language": "en-US,en;q=0.9"
    }

    try:
        response = requests.get(index_url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        table = soup.find("table", class_="tableFile")
        if table:
            for row in table.find_all("tr"):
                cols = row.find_all("td")
                if len(cols) >= 3:
                    a_tag = cols[2].find("a")
                    doc_type = cols[3].text.strip().lower()
                    if a_tag:
                        href = a_tag.get("href", "")
                        if href.endswith(".xml") and "form13f" in href.lower() and doc_type == "information table":
                            return "https://www.sec.gov" + href
    except Exception as e:
        print(f"Error parsing {index_url}: {e}")
    return None

**Extract the info table from the xml links**

In [7]:
def extract_13f_holdings_from_html(xml_url, filed_date, institution_name):
    headers = {
        "User-Agent": "DataScienceInternshipBot/1.0 (contact: chandanarchutha.n@gmail.com)",
        "Accept": "application/xml",
        "Accept-Language": "en-US,en;q=0.9"
    }

    response = requests.get(xml_url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    tables = soup.find_all("table")
    if not tables:
        return []

    data_table = tables[-1]
    rows = data_table.find_all("tr")
    holdings = []

    for row in rows[4:]:
        cols = row.find_all("td")
        if len(cols) >= 13:
            issuer = cols[0].text.strip()
            class_title = cols[1].text.strip()
            cusip = cols[2].text.strip()
            value = cols[4].text.strip()
            shares = cols[5].text.strip()
            discretion = cols[8].text.strip()
            voting_sole = cols[10].text.strip()
            voting_shared = cols[11].text.strip()
            voting_none = cols[12].text.strip()

            holdings.append([
                issuer, class_title, cusip, value, shares, discretion,
                voting_sole, voting_shared, voting_none, filed_date, xml_url, institution_name
            ])
    return holdings


**Fetching the data at once for everything**

In [8]:
# without the time thing, the wait between requests

all_holdings = []

for cik, institution_name in top_institutions.items():
    filing_links, filed_dates, institution_names = fetch_filing_metadata(cik, institution_name)
    xml_links = [extract_info_table_xml(link) for link in filing_links]
    sample_filings = list(zip(xml_links, filed_dates, institution_names))

    for xml_url, filed_date, institution_name in sample_filings:
        if xml_url:
            try:
                extracted = extract_13f_holdings_from_html(xml_url, filed_date, institution_name)
                all_holdings.extend(extracted)
            except Exception as e:
                print(f"Error parsing {xml_url}: {e}")

Error parsing https://www.sec.gov/Archives/edgar/data/354204/000035420422001931/xslForm13F_X01/Filing13F_Filtered_Q12022.xml: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


In [ ]:

# with time thing. there is a pause between each request

import time

# Final version of scraping loop with improvements
all_holdings = []
failed_urls = []

for cik, institution_name in top_institutions.items():
    filing_links, filed_dates, institution_names = fetch_filing_metadata(cik, institution_name)
    xml_links = [extract_info_table_xml(link) for link in filing_links]
    sample_filings = list(zip(xml_links, filed_dates, institution_names))
    for i, (xml_url, filed_date, institution_name) in enumerate(sample_filings):
        if xml_url:
            try:
                extracted = extract_13f_holdings_from_html(xml_url, filed_date, institution_name)
                all_holdings.extend(extracted)
                
                time.sleep(0.5)  # polite pause between requests to avoid SEC throttling
            except Exception as e:
                print(f"Error parsing {xml_url}: {e}")
                failed_urls.append((xml_url, institution_name))



    
if failed_urls:
    print(f"\n❌ {len(failed_urls)} filings failed to parse. You can retry them later if needed.")
    for url, name in failed_urls:
        print(f"- {name}: {url}")



In [9]:
data = pd.DataFrame(all_holdings, columns=[
    "Issuer", "Class", "CUSIP", "Value (x$1000)", "Shares", "Discretion",
    "Voting - Sole", "Voting - Shared", "Voting - None", "Filed Date", "Source URL", "Institution"
])        


In [10]:
data.to_csv('13F_filings.csv', index=False)

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17451 entries, 0 to 17450
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Issuer           17451 non-null  object
 1   Class            17451 non-null  object
 2   CUSIP            17451 non-null  object
 3   Value (x$1000)   17451 non-null  object
 4   Shares           17451 non-null  object
 5   Discretion       17451 non-null  object
 6   Voting - Sole    17451 non-null  object
 7   Voting - Shared  17451 non-null  object
 8   Voting - None    17451 non-null  object
 9   Filed Date       17451 non-null  object
 10  Source URL       17451 non-null  object
 11  Institution      17451 non-null  object
dtypes: object(12)
memory usage: 1.6+ MB


In [14]:
data.head(10)

,Issuer,Class,CUSIP,Value (x$1000),Shares,Discretion,Voting - Sole,Voting - Shared,Voting - None,Filed Date,Source URL,Institution
0,ALLY FINL INC,COM,02005N100,"102,257,321","2,803,875",DFND,"2,803,875",0,0,2025-05-15T16:06:43-04:00,https://www.sec.gov/Archives/edgar/data/106798...,Berkshire Hathaway
1,ALLY FINL INC,COM,02005N100,"154,202,454","4,228,200",DFND,"4,228,200",0,0,2025-05-15T16:06:43-04:00,https://www.sec.gov/Archives/edgar/data/106798...,Berkshire Hathaway
2,ALLY FINL INC,COM,02005N100,"114,406,390","3,137,000",DFND,"3,137,000",0,0,2025-05-15T16:06:43-04:00,https://www.sec.gov/Archives/edgar/data/106798...,Berkshire Hathaway
3,ALLY FINL INC,COM,02005N100,"176,378,038","4,836,250",DFND,"4,836,250",0,0,2025-05-15T16:06:43-04:00,https://www.sec.gov/Archives/edgar/data/106798...,Berkshire Hathaway
4,ALLY FINL INC,COM,02005N100,"46,499,250","1,275,000",DFND,"1,275,000",0,0,2025-05-15T16:06:43-04:00,https://www.sec.gov/Archives/edgar/data/106798...,Berkshire Hathaway
5,AMAZON COM INC,COM,023135106,"1,469,568,240","7,724,000",DFND,"7,724,000",0,0,2025-05-15T16:06:43-04:00,https://www.sec.gov/Archives/edgar/data/106798...,Berkshire Hathaway
6,AMAZON COM INC,COM,023135106,"433,031,760","2,276,000",DFND,"2,276,000",0,0,2025-05-15T16:06:43-04:00,https://www.sec.gov/Archives/edgar/data/106798...,Berkshire Hathaway
7,AMERICAN EXPRESS CO,COM,025816109,"309,391,895","1,149,942",DFND,"1,149,942",0,0,2025-05-15T16:06:43-04:00,https://www.sec.gov/Archives/edgar/data/106798...,Berkshire Hathaway
8,AMERICAN EXPRESS CO,COM,025816109,"40,104,874,157","149,061,045",DFND,"149,061,045",0,0,2025-05-15T16:06:43-04:00,https://www.sec.gov/Archives/edgar/data/106798...,Berkshire Hathaway
9,AMERICAN EXPRESS CO,COM,025816109,"376,592,783","1,399,713",DFND,"1,399,713",0,0,2025-05-15T16:06:43-04:00,https://www.sec.gov/Archives/edgar/data/106798...,Berkshire Hathaway


Even though all 5 rows are about ALLY FINL INC, each one refers to:

- A different block of shares
- Different voting authority configurations
- Potentially different internal managers at Berkshire handling that position
